In [5]:
import sys
import os
sys.path.append("..")
from globals import ROOT_DIR
import configparser
import csv
import pandas as pd
import numpy as np 
import tweepy
config = configparser.ConfigParser()
config.read('../config.ini')
consumer_key = config['DEFAULT']['TWITTER_CONSUMER_KEY']
consumer_secret_key = config['DEFAULT']['TWITTER_CONSUMER_SECRET_KEY']
access_token = config['DEFAULT']['TWITTER_ACCESS_TOKEN']
access_token_secret = config['DEFAULT']['TWITTER_ACCESS_TOKEN_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


In [3]:
data_file = os.path.join(ROOT_DIR, 'data/founta_data.npy')
data = np.load(data_file)

In [4]:
data = data[()]

In [89]:
users = [value['user']['id'] for key, value in data.items()]

In [90]:
len(users)

64149

In [38]:
count = 0 
for key, value in data.items():
    if count > 2:
        break
    tweet = value
    count += 1
tweet

{'created_at': 'Sun Apr 02 22:49:05 +0000 2017',
 'id': 848668638869671939,
 'id_str': '848668638869671939',
 'text': 'Damn dean just put Corbin to sleep. That Match Also Showed why It Was On Pre show. Boring as fuck',
 'truncated': False,
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 3308335625,
  'id_str': '3308335625',
  'name': 'Sam',
  'screen_name': 'Samiam3187',
  'location': 'NC',
  'description': 'Panthers , Hornets , Yankees , Blue Devils, Wrestling, MMA, TV, Music, Movies\n\n#KeepPounding',
  'url': None,
  'entities': {'description': {'urls': []}},
  'protected': False,
  'followers_count': 1288,
  'friends_count': 2122,
  'listed_count': 12,
  'created_at': 'Thu Ju

In [39]:
user_id = tweet['user']['id_str']
user_id

'3308335625'

In [13]:
obj = api.user_timeline(user_id)

In [20]:
obj[0]._json

{'created_at': 'Fri Jul 12 13:23:05 +0000 2019',
 'id': 1149670514383679489,
 'id_str': '1149670514383679489',
 'text': 'RT @a_hiraeth: !!! Share peeps https://t.co/9xleDH6mQ0',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'a_hiraeth',
    'name': '✨𝖓𝖊𝖜 𝖒𝖆𝖌𝖎𝖈 𝖜𝖆𝖓𝖉✨',
    'id': 2956587487,
    'id_str': '2956587487',
    'indices': [3, 13]}],
  'urls': [],
  'media': [{'id': 1149448726819115008,
    'id_str': '1149448726819115008',
    'indices': [31, 54],
    'media_url': 'http://pbs.twimg.com/media/D_OphumUEAA6Eqp.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/D_OphumUEAA6Eqp.jpg',
    'url': 'https://t.co/9xleDH6mQ0',
    'display_url': 'pic.twitter.com/9xleDH6mQ0',
    'expanded_url': 'https://twitter.com/a_hiraeth/status/1149448729650315265/photo/1',
    'type': 'photo',
    'sizes': {'large': {'w': 1080, 'h': 1075, 'resize': 'fit'},
     'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'medium': {'w': 1080,

In [23]:
status_ids = [item._json['id'] for item in obj]
oldest = status_ids[-1]

In [55]:
obj3 = api.user_timeline(user_id, count = 3300)

In [60]:
oldest = obj3[0]._json['id']
oldest

1149646421701693441

In [65]:
obj3[-1]._json['created_at']
obj3[-1]._json['id']

1145118435812282368

In [66]:
obj4 = api.user_timeline(user_id, count = 3300, since_id = oldest)

In [68]:
len(obj4)

0

In [45]:
obj2[-1]._json

{'created_at': 'Sat Jun 29 23:54:45 +0000 2019',
 'id': 1145118435812282368,
 'id_str': '1145118435812282368',
 'text': 'I hate light spots. Can we end them please',
 'truncated': False,
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 3308335625,
  'id_str': '3308335625',
  'name': 'Sam',
  'screen_name': 'Samiam3187',
  'location': 'NC',
  'description': 'Panthers , Hornets , Yankees , Blue Devils, Wrestling, MMA, TV, Music, Movies\n\n#KeepPounding',
  'url': None,
  'entities': {'description': {'urls': []}},
  'protected': False,
  'followers_count': 1288,
  'friends_count': 2121,
  'listed_count': 12,
  'created_at': 'Thu Jun 04 16:03:04 +0000 2015',
  'favourites_count': 1564

196


In [120]:
import time 
# user_id = {}
user_data = np.load(os.path.join(ROOT_DIR, 'data/user_data.npy'))
user_data = user_data[()]
start = time.time()
for i, user_id in enumerate(users):
    print("On user {}".format(i))
    page_count = 0 
#     for pages in tweepy.Cursor(api.user_timeline, id=user_id, count=200).pages(): 
    tweets = api.user_timeline(user_id, count = 200)
    user_data[user_id] = [tweet._json['text'] for tweet in tweets] #each user has list of text tweets 
#     for item in pages:
#               user_data[user_id].append(item._json['text'])
        
    np.save(os.path.join(ROOT_DIR, 'data/user_data.npy'), user_data)
#     print(time.time() - start)

On user 0
On user 1
On user 2
On user 3
On user 4
On user 5
On user 6
On user 7
On user 8
On user 9
On user 10
On user 11
On user 12
On user 13
On user 14
On user 15
On user 16
On user 17
On user 18
On user 19
On user 20
On user 21
On user 22
On user 23
On user 24
On user 25
On user 26
On user 27
On user 28
On user 29
On user 30
On user 31
On user 32
On user 33
On user 34
On user 35
On user 36
On user 37
On user 38
On user 39
On user 40
On user 41
On user 42
On user 43
On user 44
On user 45
On user 46
On user 47
On user 48
On user 49
On user 50
On user 51


KeyboardInterrupt: 

In [119]:
for key, value in user_data.items():
    print(value)
    break

196


3227

In [97]:
from collections import defaultdict


#make it blank 
user_data = defaultdict(list)
np.save(os.path.join(ROOT_DIR, 'data/user_data.npy'), user_data)

for user_id in users:
    since_id = None 
    rate_limit = 200
    start_ptr = 0
    end_ptr = start_ptr + rate_limit
    while(True):
        print("Start ptr is at {}".format(start_ptr))

        #load next series of tweets 
        if since_id is not None:
            prev_tweets = tweets 
            if len(tweets) == 0:
                print("Total length of tweets collected: {}".format(len(user_data[user_id])))
                break 
        else:
            tweets = api.user_timeline(user_id, count = rate_limit)
            if len(tweets) == 0:
                print("Total length of tweets collected: {}".format(len(tweets)))
                break 

        since_id = tweets[0]._json['id']

        # load and save results each time 
#         if os.path.exists(os.path.join(ROOT_DIR, 'data/user_data.npy')):
#             user_data = np.load(os.path.join(ROOT_DIR, 'data/user_data.npy'))
#             user_data = user_data[()]
#         else: 
#             user_data = defaultdict(list)
#         print(len(tweets)) 
        for i, tweet in enumerate(tweets):
            tweet = tweet._json
            user_data[user_id].append(tweet['text'])
        np.save(os.path.join(ROOT_DIR, 'data/user_data.npy'), user_data)

        start_ptr += rate_limit 
        end_ptr += rate_limit


Start ptr is at 0
Start ptr is at 200
Total length of tweets collected: 196
Start ptr is at 0
Start ptr is at 200
Total length of tweets collected: 199
Start ptr is at 0
Start ptr is at 200
Total length of tweets collected: 200
Start ptr is at 0
Start ptr is at 200
Total length of tweets collected: 200
Start ptr is at 0
Start ptr is at 200
Total length of tweets collected: 200
Start ptr is at 0
Start ptr is at 200
Total length of tweets collected: 199
Start ptr is at 0
Start ptr is at 200
Total length of tweets collected: 200
Start ptr is at 0
Start ptr is at 200


KeyboardInterrupt: 

In [87]:
for key, value in user_data.items():
    print(len(value))

200


dict_keys(['3308335625'])

In [88]:
tweets = api.user_timeline('3308335625', count = 3300)
len(tweets)

200